In [35]:
from sklearn import svm,metrics
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier #분류
from sklearn.neighbors import KNeighborsRegressor  #회귀 예측
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler #표준화작업
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import requests
import oracledb
import random
import glob,os.path,re
import seaborn as sns
matplotlib.rcParams['axes.unicode_minus'] = False  # 마이너스 표시 해결
# 한글설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # windows 사용자
# matplotlib.rcParams['font.family'] = 'AppleGothic Gothic' # Mac사용자
matplotlib.rcParams['font.size'] = '10' # 글자크기

In [36]:
import pandas as pd
df = pd.read_csv('https://bit.ly/fish_csv')
df.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


1. 전처리

In [37]:
# 1. 데이터 전처리 - 데이터 확인 후 data, target분리
f_target=df['Species'].to_numpy()
f_input= df.iloc[:,1:].to_numpy()

# train, test 분리
train_input,test_input,train_target,test_target = train_test_split(
    f_input,f_target,random_state=42
)

In [38]:
# 표준화 작업
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

2. 실습 /  C 규제: 규제가 올라가면 규제가 약해지고, 낮아지면 규제가 강해짐, 선형회귀 규제 - alpha 값으로 지정

실습의 기존방법에서 결과값이 떨어지면 사용

In [39]:
from sklearn.linear_model import LogisticRegression
# max_iter: 반복횟수 = 100 -> 기본설정 
# C 규제: 규제가 올라가면 규제가 약해지고, 낮아지면 규제가 강해짐, 선형회귀 규제 - alpha 값으로 지정

lr = LogisticRegression(C=30, max_iter=1000)
lr.fit(train_scaled,train_target)

LogisticRegression(C=30, max_iter=1000)

2.실습 / 기존

In [40]:
# # 데이터 실습
# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression()
# lr.fit(train_scaled,train_target)


3. 예측하기

In [41]:
# 예측하기
predict = lr.predict(test_scaled[:5])
predict

array(['Perch', 'Smelt', 'Pike', 'Roach', 'Perch'], dtype=object)

4. 정답률

In [42]:
lr.score(test_scaled,test_target)

0.925

In [43]:
lr.score(train_scaled,train_target)


0.9663865546218487

In [49]:
lr.predict(test_scaled[:5])

array(['Perch', 'Smelt', 'Pike', 'Roach', 'Perch'], dtype=object)

In [50]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.    0.009 0.882 0.    0.103 0.004 0.002]
 [0.    0.001 0.032 0.    0.005 0.962 0.   ]
 [0.    0.    0.024 0.927 0.008 0.041 0.   ]
 [0.007 0.032 0.256 0.005 0.617 0.    0.084]
 [0.    0.    0.927 0.002 0.068 0.002 0.001]]


proba

predict probability, 즉 각 클래스의 확률을 예측하라는 의미, 
결과값 도출 분석 

5번

In [44]:
# [[0.004 0.091 0.545 0.008 0.28  0.061 0.01 ] -> 4변의 이 결과를 가지고 perch가 결정됨
# z점수를 출력하여 softmax에서 가장 높은 확률로 아래 결과가 나옴
predict=lr.predict(test_scaled[:5])
print(predict)

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


1번

In [45]:
lr.classes_
# 아래 숫자 array 순

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

2번

In [46]:
proba = lr.predict_proba(test_scaled[:5])
np.round(proba, decimals=3)

array([[0.   , 0.009, 0.882, 0.   , 0.103, 0.004, 0.002],
       [0.   , 0.001, 0.032, 0.   , 0.005, 0.962, 0.   ],
       [0.   , 0.   , 0.024, 0.927, 0.008, 0.041, 0.   ],
       [0.007, 0.032, 0.256, 0.005, 0.617, 0.   , 0.084],
       [0.   , 0.   , 0.927, 0.002, 0.068, 0.002, 0.001]])

3번

In [47]:
# 위의 결과로 z점수를 만듦
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[-7.500e+00  1.110e+00  5.690e+00 -2.780e+00  3.540e+00  3.700e-01
  -4.300e-01]
 [-1.244e+01  1.940e+00  5.190e+00 -2.300e+00  3.290e+00  8.590e+00
  -4.280e+00]
 [-5.120e+00 -7.230e+00  3.410e+00  7.060e+00  2.370e+00  3.940e+00
  -4.430e+00]
 [-9.400e-01  5.700e-01  2.660e+00 -1.310e+00  3.540e+00 -6.070e+00
   1.550e+00]
 [-7.440e+00 -2.280e+00  6.340e+00  1.000e-02  3.730e+00  3.200e-01
  -6.900e-01]]


4번

In [48]:
# 위의 행 하나마다 1로 만들어 주는 것
from scipy.special import softmax
proba = softmax(decision, axis= 1)
print(np.round(proba, decimals=3)) # 행 하나하나 1로 맞춤
# 결정하는 것이 많으면 

[[0.    0.009 0.882 0.    0.103 0.004 0.002]
 [0.    0.001 0.032 0.    0.005 0.962 0.   ]
 [0.    0.    0.024 0.927 0.008 0.041 0.   ]
 [0.007 0.032 0.256 0.005 0.617 0.    0.084]
 [0.    0.    0.927 0.002 0.068 0.002 0.001]]
